In [1]:
import sqlite3
import csv
import pandas as pd

In [2]:
try:
    with open('Applause_schema_lite.sql') as file:
        create_db = file.read()
except FileNotFoundError:
    print('Did not find file.')

In [3]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [4]:
cur.executescript(create_db)
con.commit()

del create_db

### Place data from CSV into Table
Note: This can be a long and tendious process.

In [5]:
with open('../Applause_files/testers.csv') as csv_file:
    csv_reader = list(csv.DictReader(csv_file))
    to_db = [(i['testerId'], i['firstName'], i['lastName'], i['country'], i['lastLogin']) for i in csv_reader]
        
cur.executemany("INSERT INTO testers (testerId, firstName, lastName, country, lastLogin) VALUES (?, ?, ?, ?, ?);", to_db)
con.commit()

with open('../Applause_files/devices_devicename.csv') as csv_file:
    csv_reader = list(csv.DictReader(csv_file))
    to_db = [(i['deviceId'], i['deviceName']) for i in csv_reader]
        
cur.executemany("INSERT INTO devices (deviceId, deviceName) VALUES (?, ?);", to_db)
con.commit()

with open('../Applause_files/tester_device.csv') as csv_file:
    csv_reader = list(csv.DictReader(csv_file))
    to_db = [(i['testerId'], i['deviceId']) for i in csv_reader]
        
cur.executemany("INSERT INTO tester_device (testerId, deviceId) VALUES (?, ?);", to_db)
con.commit()

with open('../Applause_files/bugs.csv') as csv_file:
    csv_reader = list(csv.DictReader(csv_file))
    to_db = [(i['bugId'], i['testerId'], i['deviceId']) for i in csv_reader]
        
cur.executemany("INSERT INTO bugs (bugId, testerId, deviceId) VALUES (?, ?, ?);", to_db)
con.commit()

del csv_reader

### The Pandas Method
Note: Only works if the data can be fit into memory.

In [6]:
df = pd.read_csv('../Applause_files/testers.csv')
df.to_sql('testers', con, if_exists='replace', index=False)

df = pd.read_csv('../Applause_files/devices_devicename.csv')
df.to_sql('devices', con, if_exists='replace', index=False)

df = pd.read_csv('../Applause_files/tester_device.csv')
df.to_sql('tester_device', con, if_exists='replace', index=False)

df = pd.read_csv('../Applause_files/bugs.csv')
df.to_sql('bugs', con, if_exists='replace', index=False)

del df

In [7]:
query1 = """
SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM testers 
INNER JOIN bugs
ON testers.testerId = bugs.testerId
GROUP BY  firstName, lastName
ORDER BY COUNT(bugId) DESC;
"""

cur.execute(query1)

rows = cur.fetchall()

for row in rows:
    print(row)

('Taybin', 'Rutkin', 125)
('Lucas', 'Lowry', 117)
('Sean', 'Wellington', 116)
('Miguel', 'Bautista', 114)
('Stanley', 'Chen', 110)
('Mingquan', 'Zheng', 109)
('Leonard', 'Sutton', 106)
('Darshini', 'Thiagarajan', 104)
('Michael', 'Lubavin', 99)


In [8]:
query2 = """
SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM bugs 
INNER JOIN testers
ON bugs.testerid = testers.testerid
INNER JOIN devices
ON bugs.deviceId = devices.deviceId
WHERE deviceName IN ('iPhone 4', 'iPhone 5') /* case sensitive */
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;
"""

cur.execute(query2)

rows = cur.fetchall()

for row in rows:
    print(row)

('Stanley', 'Chen', 110)
('Taybin', 'Rutkin', 66)
('Sean', 'Wellington', 58)
('Miguel', 'Bautista', 53)
('Leonard', 'Sutton', 32)
('Mingquan', 'Zheng', 21)


In [9]:
query3 = """
SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM testers
INNER JOIN bugs
ON testers.testerId = bugs.testerId
WHERE country IN ('US', 'JP') /* case sensitive */
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;
"""

cur.execute(query3)

rows = cur.fetchall()

for row in rows:
    print(row)

('Taybin', 'Rutkin', 125)
('Lucas', 'Lowry', 117)
('Sean', 'Wellington', 116)
('Miguel', 'Bautista', 114)
('Mingquan', 'Zheng', 109)
('Michael', 'Lubavin', 99)


In [10]:
query4 = """
SELECT firstName, lastName, COUNT(bugId) AS bugs_found
FROM bugs
INNER JOIN testers
ON bugs.testerId = testers.testerId
INNER JOIN devices
ON bugs.deviceId = devices.deviceId
WHERE country IN ('US', 'JP') /* case sensitive */ AND devicename IN ('iPhone 4', 'iPhone 5') /* case sensitive */
GROUP BY firstName, lastName
ORDER BY COUNT(bugId) DESC;
"""

cur.execute(query4)

rows = cur.fetchall()

for row in rows:
    print(row)

('Taybin', 'Rutkin', 66)
('Sean', 'Wellington', 58)
('Miguel', 'Bautista', 53)
('Mingquan', 'Zheng', 21)
